In [ ]:
import tensorflow as tf
import horovod.tensorflow.keras as hvd
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from itertools import chain
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

total_partitions = hvd.size()


#Reading the target labels
df = pd.read_csv('./sample/sample_labels.csv', index_col= False)
df.index.astype('int64')
total_rows = df.shape[0]
batch_size = total_rows//total_partitions

for partition in range(total_partitions):
    if hvd.rank() == partition:
        start = partition * batch_size
        end = (partition+1) * batch_size
        df = df.iloc[start:end-1, :]

all_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
 'Pneumonia', 'Pneumothorax','No Finding']

#Making the labels as column and assigning 1 if it exists in Finding Column else 0
for c_label in all_labels:
    if len(c_label)>1:
        df[c_label] = df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)

# Reading Images into arrays
train_path = './sample/sample/images/'
data = os.listdir(train_path)
train_data=[]
imagenames_list = []
i =0

for partition in range(total_partitions):
    if hvd.rank() == partition:
        start = partition * batch_size
        end = (partition+1) * batch_size
        data= data[start:end-1]
for img in tqdm(data):
    img_array = os.path.join(train_path,img)
    imagenames_list.append(img_array)

df['path'] = imagenames_list

train_df, test_df = train_test_split(df,test_size = 0.20,random_state = 2018)

IMG_SIZE = (100, 100)
datagen = ImageDataGenerator(samplewise_center=True,
                              samplewise_std_normalization=True,
                              horizontal_flip = True,
                              vertical_flip = False,
                              height_shift_range= 0.05,
                              width_shift_range=0.1,
                              rotation_range=5,
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)

train_generator = datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=None,
            x_col="path",
            y_col= ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
 'Pneumonia', 'Pneumothorax','No Finding'],
            subset="training",
            batch_size=256,
            seed=42,
            shuffle=True,
            target_size= IMG_SIZE,rescale=1.0/255.0, color_mode='grayscale',class_mode = 'raw')

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
dataframe=test_df,
directory=None,
x_col="path",
y_col=['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
 'Pneumonia', 'Pneumothorax', 'No Finding'],
class_mode="raw",
batch_size=1024,
seed=42,
shuffle=False,
target_size=IMG_SIZE,rescale=1.0/255.0, color_mode='grayscale')

model = tf.keras.Sequential([
tf.keras.layersConv2D(filters=96, kernel_size=(11, 11),input_shape =(128,128,1),strides=4, kernel_regularizer=l2(0.)),
tf.keras.layers.Activation('relu'),
tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
tf.keras.layers.BatchNormalization(),

#Layer 2
tf.keras.layers.Conv2D(256, (5, 5), padding='same'),
tf.keras.layers.Activation('relu'),
tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
tf.keras.layers.BatchNormalization(),

# Layer 3
tf.keras.layers.Conv2D(384, (3, 3), padding='same'),
tf.keras.layers.Activation('relu'),

#Layer 4
tf.keras.layers.Conv2D(384, (3, 3), padding='same'),
tf.keras.layers.Activation('relu'),

# Layer 5
tf.keras.layers.Conv2D(256, (3, 3), padding='same'),
tf.keras.layers.Activation('relu'),
tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),

# Layer 6
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(4096),
tf.keras.layers.Activation('relu'),
tf.keras.layers.Dropout(0.5),

# Layer 7
tf.keras.layers.Dense(4096),
tf.keras.layers.Activation('relu'),
tf.keras.layers.Dropout(0.5),

# Layer 8
tf.keras.layers.Dense(15, activation='softmax')
])

# Horovod: adjust learning rate based on number of GPUs.
opt = tf.optimizers.Adam(0.001 * hvd.size())

# Horovod: add Horovod DistributedOptimizer.
opt = hvd.DistributedOptimizer(opt)

# Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
# uses hvd.DistributedOptimizer() to compute gradients.
model.compile(loss=tf.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy'],
                    experimental_run_tf_function=False)

callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    #
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first three epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=3, verbose=1),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
#if hvd.rank() == 0:
#    callbacks.append(tf.keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))

# Horovod: write logs on worker 0.
verbose = 1 if hvd.rank() == 0 else 0

# Train the model.
# Horovod: adjust number of steps based on number of GPUs.
history = history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator),
                                        validation_data=test_generator,epochs=10, verbose=1)

(loss, accuracy) = model.evaluate_generator(test_generator,verbose=1)
print('[INFO] accuracy: {:.2f}%'.format(accuracy * 100))

predicted = model.predict_generator(test_generator, steps = len(test_generator), verbose = True)

from matplotlib import pyplot as plt
def summarize_diagnostics(history):
    figure,ax = plt.subplots()
    plt.figure(figsize=(10,10))
# plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    figure.tight_layout(pad=3.0)

summarize_diagnostics(history)

test_labels = []
for i in range(0,5):
    test_labels.extend(np.array(test_generator[i][1]))
len(test_labels)


test_x,test_y = next(test_generator)

for c_label, p_count, t_count in zip(all_labels, 
                                     100*np.mean(test_labels,0), 
                                     100*np.mean(predicted,0)):
    print('%s: Dx: %2.2f%%, PDx: %2.2f%%' % (c_label, t_count, p_count))
    
    
    
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_labels[:,idx].astype(int), predicted[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('barely_trained_net.png')


sickest_idx = np.argsort(np.sum(test_y, 1)<1)
fig, m_axs = plt.subplots(4, 3, figsize = (10, 15))
for (idx, c_ax) in zip(sickest_idx, m_axs.flatten()):
    c_ax.imshow(test_x[idx, :,:,0], cmap = 'bone')
    stat_str = [n_class[:6] for n_class, n_score in zip(all_labels, 
                                                                  test_labels[idx]) 
                             if n_score>0.5]
    pred_str = ['%s:%2.0f%%' % (n_class[:4], p_score*100)  for n_class, n_score, p_score in zip(all_labels, 
                                                                  test_labels[idx], predicted[idx]) 
                             if (n_score>0.5) or (p_score>0.5)]
    c_ax.set_title('Dx: '+', '.join(stat_str)+'\nPDx: '+', '.join(pred_str))
    c_ax.axis('off')
fig.savefig('trained_img_predictions.png')